In [1]:
!pip install -q langchain pypdf2 tiktoken textract faiss-cpu huggingface_hub pypdfium2 InstructorEmbedding sentence-transformers python-docx contractions -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 80.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.9/106.9 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 14.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install qdrant-client

In [14]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
from qdrant_client.http.models import CollectionStatus

In [2]:
from transformers import AutoModel, AutoTokenizer
import torch
import os
import contractions
import pypdfium2 as pdfium
import re
import glob

In [3]:
tokenizer_de = AutoTokenizer.from_pretrained("thenlper/gte-base")
model_de = AutoModel.from_pretrained("thenlper/gte-base")

In [4]:
from torch import Tensor
def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
  last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
  return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

In [31]:
def embedding(text):
  batch_dict = tokenizer_de(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
  outputs = model_de(**batch_dict)
  embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
  embeddings = embeddings.detach().numpy()
  embeddings = embeddings[0]
  return embeddings

In [6]:
def pdfium_get_text(data: bytes) -> str:
    output = ""
    pdf = pdfium.PdfDocument(data)
    for i in range(len(pdf)):
        page = pdf.get_page(i)
        textpage = page.get_textpage()
        text = textpage.get_text_range()
        output+=text + "\n"
    return output

In [24]:
def pdf_2_snapshot(pdf):
  with open(pdf,"rb") as f:
    data = f.read()
    entire_text=pdfium_get_text(data)
    canton = pdf.split('/')[-1].split('_')[0]
    file_name = pdf.split('/')[-1]
    text_embedding = embedding(entire_text)
  return entire_text,canton,file_name,text_embedding

In [ ]:
pdf_files =glob.glob('/content/*.pdf')
text_embedding_lst, payload_lst= [],[]
base_url = 'https://entscheidsuche.ch/docs/'
for pdf in pdf_files:
  text,canton,file_name,text_embedding = pdf_2_snapshot(pdf)
  payload_dict = {}
  payload_dict['content'] = text
  payload_dict['canton'] = canton
  full_url = base_url+canton+'_Gerichte/'+file_name
  payload_dict['link'] = full_url
  payload_lst.append(payload_dict)

In [35]:
len(text_embedding_lst[0])

768

In [37]:
client = QdrantClient(host="localhost", port=6333)

In [ ]:
my_collection = "first_collection"

first_collection = client.recreate_collection(
    collection_name=my_collection,
    vectors_config=models.VectorParams(size=768, distance=models.Distance.COSINE)
)
print(first_collection)

In [ ]:
client.upsert(
    collection_name=my_collection,
    points=models.Batch(
        ids=[1],
        vectors=text_embedding_lst
        payloads = payload_lst
    )
)

In [ ]:
snapshot_info = client.create_snapshot(collection_name=collection_name)
print(snapshot_info)